In [76]:
import cv2
import numpy as np
import os
import copy
from matplotlib import pyplot as plt
from scipy import ndimage as ndi
from skimage.morphology import watershed
from skimage.feature import peak_local_max
from sklearn.cluster import MeanShift
from random import randint
#from detector import Detectors
from scipy.spatial import distance as dist
import collections
from collections import OrderedDict
from random import randint

In [77]:
class CentroidTracker():
    def __init__(self, maxDisappeared=50):
        self.nextId = 0
        #self.vid = vid
        #self.windows = windows
        self.objects=OrderedDict()
        self.disappeared=OrderedDict()
        self.maxDisappeared=maxDisappeared 
        #self.trajectory=[self.objects[self.nextId]]
        #self.tracker = self._init_tracker(windows, frame)
    def register(self,center):
        self.objects[self.nextId]=center
        self.disappeared[self.nextId]=0
        self.nextId+=1
    def deregister(self,objectID):
        del self.objects[objectID]
        del self.disappeared[objectID]
    def update(self,cts):
        #ok, new_box = self.tracker.update(frame)
        dist_dict={}
        if len(cts)==0:
            for objectID in list(self.disappeared.keys()):
                self.disappeared[objectID]+=1
                if self.disappeared[objectID]>self.maxDisappeared:
                    self.deregister(objectID)
            return self.objects
        inputCentroids = np.zeros((len(cts), 2), dtype="int")
        for (i, (startX, startY, endX, endY)) in enumerate(cts):
            cX = int((startX + endX) / 2.0)
            cY = int((startY + endY) / 2.0)
            inputCentroids[i] = (cX, cY)
        if len(self.objects)==0:
            for i in range(0,len(inputCentroids)):
                self.register(inputCentroids[i])
        else:
            objectIDs=list(self.objects.keys())
            objectCenters=list(self.objects.values())
            D = dist.cdist(np.array(objectCenters), inputCentroids)
            #dist_dict={}
            #dist_dict[self.objects.keys()]=D
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            usedRows=set()
            usedCols=set()
            for (row, col) in zip(rows, cols):
                if row in usedRows or col in usedCols:
                    continue
                objectID=objectIDs[row]
                self.objects[objectID]=inputCentroids[col]
                self.disappeared[objectID]=0
                usedRows.add(row)
                usedCols.add(col)
            unusedRows=set(range(0,D.shape[0])).difference(usedRows)
            unusedCols=set(range(0,D.shape[1])).difference(usedCols)
            if D.shape[0] >= D.shape[1]:
                for row in unusedRows:
                    objectID = objectIDs[row]
                    self.disappeared[objectID] += 1
                    if self.disappeared[objectID]>self.maxDisappeared:
                        self.deregister(objectID)
            else:
                for col in unusedCols:
                    self.register(inputCentroids[col])
        return self.objects

In [78]:
class Features():
   
    def centercells(self,frame):
        #contours2=contours_cells(frame)
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
        contours1=[] 
        centers=[]
        for i in contours:
            #x, y, w, h = cv2.boundingRect(i)
            if cv2.contourArea(i)>25:  #Remove small connected areas
                #x, y, w, h = cv2.boundingRect(i)
                contours1.append(i)
        #centers=[]
                for m,n in zip(contours1,range(len(contours1))):
                    M = cv2.moments(m)
                    cX=int(M["m10"]/M["m00"])
                    cY=int(M["m01"]/M["m00"])
                    #arr=np.array((cX,cY))
                    centers.append((cX,cY))
        #D=dist.cdist()
        return centers
    def perimetercells(self,frame):
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
        contours1=[] 
        plength=[]
        for i in contours:
            #x, y, w, h = cv2.boundingRect(i)
            if cv2.contourArea(i)>25:  #Remove small connected areas
                #x, y, w, h = cv2.boundingRect(i)
                contours1.append(i)
        #contours3=contours_cells(frame)
        #plength=[]
                for m,n in zip(contours1,range(len(contours1))):
                    M = cv2.moments(m)
                    plength.append(cv2.arcLength(m,True))
        return plength
    
    def areacells(self,frame):
        #contours4=contours_cells(frame)
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
        contours1=[] 
        areas=[]
        for i in contours:
            #x, y, w, h = cv2.boundingRect(i)
            if cv2.contourArea(i)>25:  #Remove small connected areas
                #x, y, w, h = cv2.boundingRect(i)
                contours1.append(i)
                for m,n in zip(contours1,range(len(contours1))):
                    M = cv2.moments(m)
                    areas.append(cv2.contourArea(m))
        return areas
    def ellipsecells(self,frame):
        pi=3.1415926
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
        contours1=[] 
        P,A,Q=[],[],[]
        for i in contours:
            #x, y, w, h = cv2.boundingRect(i)
            if cv2.contourArea(i)>25:  #Remove small connected areas
                #x, y, w, h = cv2.boundingRect(i)
                contours1.append(i)
                for m,n in zip(contours1,range(len(contours1))):
                    M = cv2.moments(m)
                    ellipse=cv2.fitEllipse(m)
                    #ellipse[1][0]
                    P.append(int(ellipse[1][0]*pi+2*(ellipse[1][1]-ellipse[1][0])))
                    A.append(int(pi*ellipse[1][0]/2*ellipse[1][1]/2))
                    for l in range(len(P)):
                        #s=int(P[l]*P[l] /(4*pi * A[l]*A[l] ))
                        Q.append(int(P[l]*P[l] /(4*pi * A[l]*A[l] )))
        return Q
    def minrect_angle(self,frame):
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
        contours1=[] 
        b=[]
        for i in contours:
            #x, y, w, h = cv2.boundingRect(i)
            if cv2.contourArea(i)>25:  #Remove small connected areas
                #x, y, w, h = cv2.boundingRect(i)
                contours1.append(i)
        
                for m,n in zip(contours1,range(len(contours1))):
                    M = cv2.moments(m)
                    rect=cv2.minAreaRect(m)
                    box = cv2.boxPoints(rect)
                    box_d = np.int0(box)
                    b.append(box_d)
        return b[2]
    def rect_window(self,frame):
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
        contours1=[] 
        windows=[]
        for i in contours:
            #x, y, w, h = cv2.boundingRect(i)
            if cv2.contourArea(i)>25:  #Remove small connected areas
                #x, y, w, h = cv2.boundingRect(i)
                contours1.append(i)
                for m,n in zip(contours1,range(len(contours1))):
                    M = cv2.moments(m)
                    x,y,w,h=cv2.boundingRect(m)
                    windows.append((x,y,x+w,y+h))
        return windows
    
    def rect_center(self,frame):
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
            #Find connected domains
            #comparing the area of connected domains
        contours1=[] 
        windows_center=[]
        for i in contours:
            #x, y, w, h = cv2.boundingRect(i)
            if cv2.contourArea(i)>25:  #Remove small connected areas
                #x, y, w, h = cv2.boundingRect(i)
                contours1.append(i)
                for m,n in zip(contours1,range(len(contours1))):
                    M = cv2.moments(m)
                    x,y,w,h=cv2.boundingRect(m)
                    cx=int(x+w/2.0)
                    cy=int(y+h/2.0)
                    windows_center.append((cx,cy))
        return windows_center

In [79]:
class Detector():
    #def __init__(self):
        #self.bg=cv2.cv2.createBackgroundSubtractorMOG2()
    def Detect(self,frame,f_centers,centers):
        imgO=normalize(frame)
        imgO_1=cv2.cvtColor(imgO,cv2.COLOR_BGR2GRAY)
        ret0,thresh0 = cv2.threshold(imgO_1,0,255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
            # Deal with the situation where cells are connected together
        d,contours,hirearchy=cv2.findContours(thresh0, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        centers=[]
        r_thresh=4
        r=[]
        for i in contours:
            (x,y),radius=cv2.minEnclosingCircle(i)
            radius=int(radius)
            if radius>r_thresh:
                b=np.array((x,y))
                centers.append(np.round(b))
                r.append(radius)
        #D=dist.cdist(f_centers,centers)
        '''
        f=D<30
        for row in range(len(f_centers)):
            for c in range(len(centers)):
                if (D[row][c]<30) and (D[row][c]!=0):
                    c1=(int(centers[row][0]),int(centers[row][1]))
                    c2=(int(centers[c][0]),int(centers[c][1]))
                    r1=r[row]
                    r2=r[c]
                if (f[row][c]) and (D[row][c]!=0) and ((r[row])<16 and (r[c]<16)):
                    c1=(int(centers[row][0]),int(centers[row][1]))
                    c2=(int(centers[c][0]),int(centers[c][1]))
                    cv2.circle(frame,c1,r[row],(0,255,0),2)
                    cv2.circle(frame,c2,r[c],(0,255,0),2)
                else:
                    c1=(int(centers[row][0]),int(centers[row][1]))
                    cv2.circle(frame,c1,r[row],(0,0,255),2)
        '''        
        return centers

In [80]:
def gray_img(img):
    return cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

In [81]:
def normalize(image):
    img = image.copy().astype(np.float32)
    img -= np.mean(img)
    img /= np.linalg.norm(img)
    img = np.clip(img, 0, 255)
    img *= (1./float(img.max()))
    return (img*255).astype(np.uint8)

In [82]:
def read_img(path):
    sequence_tif = {}
    gary_seq_tif = {}
    for root, dirs, files, in os.walk(path):
        dirs.sort()
        for dir in dirs:
            sequence_tif[dir] = []
            gary_seq_tif[dir] = []
            for _, _, files_1 in os.walk(os.path.join(path, dir)):
                files_1.sort()
                for file in files_1:
                    new_dir_path = os.path.join(path, dir)
                    temp_img = cv2.imread(os.path.join(new_dir_path, file))
                    gray_imgs = gray_img(temp_img.copy())
                    sequence_tif[dir].append(temp_img)
                    gary_seq_tif[dir].append(gray_imgs)

    return sequence_tif,gary_seq_tif

In [137]:
def tracking_cells(sequence):
    detector=CentroidTracker()
    cell=Features()
    for k in range(len(sequence)):
        #former_list_centers=[]
        frame=sequence[k]
        f=copy.copy(frame)
        window=cell.rect_window(f)
        objects = detector.update(window)
        list_objects=list(objects.items())
        present_centers=cell.centercells(f)
        
        min_dist=[]
        num_present_centers=[]
        num_former_centers=[]
        match_num=[]
        match_present=[]
        
        if k==0:
            former_list_centers=cell.centercells(f)
            '''
            for item in list_objects:
                former_list_centers.append(item[1])
            '''
            for (objectID, centroid) in objects.items():
               # cv2.putText(f, text, (centroid[0] - 10, centroid[1] - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                cv2.circle(f, (centroid[0], centroid[1]), 4, (0, 255, 0), -1)
        else:
            former_list_centers=centers
            '''
            D=dist.cdist(np.array(former_list_centers),present_centers,metric='euclidean')
            D_list=D.tolist()
            for l in range(len(D_list)):
                for (n,d) in enumerate(D_list[l]):
                    if d==min(D_list[l]):
                        min_dist.append((n,d))
            for (num,(cx,cy)) in enumerate(present_centers):
                num_present_centers.append((num,(cx,cy)))
            for (num,(cx,cy)) in enumerate(former_list_centers):
                num_former_centers.append((num,(cx,cy)))
            for i in range(len(min_dist)):
                for j in range(len(num_present_centers)):
                    if min_dist[i][0]==num_present_centers[j][0]:  
                        #match_num.append(j)
                        cv2.line(f,former_list_centers[i],num_present_centers[j][1],(0,0,255),2)
            
            for i in range(len(match_num)):
                for j in range(len(num_present_centers)):
                    if match_num[i]==num_present_centers[j][0]:
                        match_present.append(num_present_centers[j][1])
            for i in range(len(former_list_centers)):
                cv2.line(f,former_list_centers[i],match_present[i],(0,0,255),2)
            '''
        centers=cell.centercells(f)
        D=dist.cdist(np.array(former_list_centers),centers,metric='euclidean') 
        for i in range(len(former_list_centers)):
            for j in range(len(centers)):
                if D[i][j]<10.0:
                    cv2.line(f,former_list_centers[i],centers[j],(0,0,255),2)
        #print(D_list)
        plt.imsave('trackingline0_'+str(k)+'.jpg',f)

In [138]:
path='/Users/admin/Downloads/COMP9517 20T2 Group Project Image Sequences-4/'

In [ ]:
path2=path+'PhC-C2DL-PSC'
original_img,grays=read_img(path2)

tracking_cells(original_img['Sequence 1'])